In [302]:
from enum import Enum
from itertools import product

import numpy as np
import pandas as pd
import pulp as P

class Task(Enum):
    VAGTHAVENDE_ELEV = 0
    ORDONNANS = 1
    UDKIG = 2
    BJÆRGEMÆRS = 3
    RORGÆNGER = 4
    UDSÆTNINGSGAST_A = 5
    UDSÆTNINGSGAST_B = 6
    UDSÆTNINGSGAST_C = 7
    UDSÆTNINGSGAST_D = 8    
    HU = 9
    PEJLEGAST_A = 10
    PEJLEGAST_B = 11
    DÆKSELEV_I_KABYS = 12

N_GASTS = 60
N_TEAMS = 3
N_SHIFTS = 6
N_TASKS = 13
MAX_SICK = 2
TEAM_SIZE = 20
TEAM_ORDER = [0,1,2]
TIDSPUNKTER = ['08 - 12', '12 - 16', '16 - 20', '20 - 24', '24 - 04', '04 - 08']

def team_minmax(team):
    return team*20, team*20+20

## Create synthetic data

In [303]:
dates = pd.date_range(start='3/1/2021', end='3/14/2021').date

history = []
np.random.seed(42)

for dt in dates:
    for shift in range(N_SHIFTS):
        n_sick = np.random.randint(0,MAX_SICK)
        sick = np.random.choice(N_GASTS, n_sick)
        team = TEAM_ORDER[shift % N_TEAMS]
        min_gast, max_gast = team_minmax(team)
        active_gasts = [gast for gast in range(min_gast, max_gast) if gast not in sick]
        for task_no, gast_no in enumerate(np.random.permutation(active_gasts)[:N_TASKS]):
            task_name = Task(task_no).name
            entry = {'date': dt, 'shift': shift, 'gast_no': gast_no, 'task_no': task_no, 'task_name': task_name}
            history.append(entry)
        for gast_no in sick:
            entry = {'date': dt, 'shift': shift, 'gast_no': gast_no, 'task_no': -1, 'task_name': 'SICK'}
            history.append(entry)
df = pd.DataFrame(history)
df.to_csv('../data/synthetic.csv', index=False)
df

,date,shift,gast_no,task_no,task_name
0,2021-03-01,0,0,0,VAGTHAVENDE_ELEV
1,2021-03-01,0,17,1,ORDONNANS
2,2021-03-01,0,15,2,UDKIG
3,2021-03-01,0,1,3,BJÆRGEMÆRS
4,2021-03-01,0,8,4,RORGÆNGER
...,...,...,...,...,...
1123,2021-03-14,5,57,9,HU
1124,2021-03-14,5,50,10,PEJLEGAST_A
1125,2021-03-14,5,44,11,PEJLEGAST_B
1126,2021-03-14,5,54,12,DÆKSELEV_I_KABYS


## Solve day

In [304]:
def solve(df, shift, sick=[], hu=[], pejleshift=3, kabystider=[0,1,2]):
    """
    TODO: handle HU
    TODO: handle vagthavende samme morgen og aften
        => udvide til at beregne alle skifter på en gang
    """
    team = TEAM_ORDER[shift % N_TEAMS]
    
    gast_min, gast_max = team_minmax(team)
    gast_range = np.arange(gast_min, gast_max)
    active = [(gast_no not in sick) for gast_no in gast_range]
    stats = df[
        (df.gast_no.isin(gast_range)) 
        & (df.task_no >= 0)
    ].groupby(['gast_no', 'task_no']).size()

    # constants
    coef = np.zeros((TEAM_SIZE, N_TASKS)).astype(int)

    for gast_no, task_no in stats.index:
        norm_gast_no = gast_no - team*TEAM_SIZE
        coef[norm_gast_no , task_no] = stats[gast_no, task_no]
    
    X = P.LpVariable.dicts('x', (range(TEAM_SIZE), range(N_TASKS)), 0, 1, P.LpBinary)

    prob = P.LpProblem('Schedule', P.LpMinimize)
    
    # objective function
    prob += P.lpSum([coef[i][j] * X[i][j] for i in range(TEAM_SIZE) for j in range(N_TASKS)])
        
    # pejlegast constraints
    pejlegast_b=None
    if shift == pejleshift:
        prob += P.lpSum([X[i][Task.PEJLEGAST_A.value] for i in range(TEAM_SIZE)]) == 1
        prob += P.lpSum([X[i][Task.PEJLEGAST_B.value] for i in range(TEAM_SIZE)]) == 1
        if pejlegast_b is not None:
            # force pejlegast b
            print('UHUHUHUHU', pejlegast_b, pejlegast_b is not None)
            prob += X[pejlegast_b][Task.PEJLEGAST_B.value] == 1
    else:
        prob += P.lpSum([X[i][Task.PEJLEGAST_A.value] for i in range(TEAM_SIZE)]) == 0
        prob += P.lpSum([X[i][Task.PEJLEGAST_B.value] for i in range(TEAM_SIZE)]) == 0


    # HU constraint (TODO)
    if len(hu):
        pass
    
    # Kabys constraint
    if shift in kabystider:
        prob += P.lpSum([X[i][Task.DÆKSELEV_I_KABYS.value] for i in range(TEAM_SIZE)]) == 1        
    else:
        prob += P.lpSum([X[i][Task.DÆKSELEV_I_KABYS.value] for i in range(TEAM_SIZE)]) == 0
    
            
    # at most one task per gast    
    for i in range(TEAM_SIZE):
        prob += P.lpSum([X[i][j] for j in range(N_TASKS)]) <= 1
        
    # do all tasks exactly once, but handle pejlegast, kabys and HU separately
    ignore = lambda j: Task(j).name.startswith('PEJLEGAST') or Task(j) == Task.DÆKSELEV_I_KABYS
    tasks = [j for j in range(N_TASKS) if not ignore(j)]     
    for j in tasks:
        prob += P.lpSum([X[i][j] for i in range(TEAM_SIZE)]) == 1
        
    # no tasks for sick gasts
    for i in sick:
        prob += P.lpSum([X[i][j] for j in range(N_TASKS)]) == 0

        
    sol = prob.solve()
    for i in range(TEAM_SIZE):
        for j in range(N_TASKS):
            x = X[i][j]
            if x.varValue == 1:
                gast_no = gast_range[i]
                task_name = Task(j)
                yield (gast_no, task_name)
    
# simulate som sick gasts
sick = [0,1,2,3]

for shift in range(N_SHIFTS):
    print(f'Vagt {TIDSPUNKTER[0]}:')
    for gast_no, task_name in solve(df, shift, sick=sick):
        print(f'- gast #{gast_no} does {task_name}')


Vagt 08 - 12:
- gast #4 does Task.VAGTHAVENDE_ELEV
- gast #6 does Task.HU
- gast #8 does Task.UDKIG
- gast #9 does Task.UDSÆTNINGSGAST_C
- gast #10 does Task.ORDONNANS
- gast #11 does Task.BJÆRGEMÆRS
- gast #13 does Task.UDSÆTNINGSGAST_A
- gast #14 does Task.DÆKSELEV_I_KABYS
- gast #16 does Task.UDSÆTNINGSGAST_B
- gast #18 does Task.RORGÆNGER
- gast #19 does Task.UDSÆTNINGSGAST_D
Vagt 08 - 12:
- gast #24 does Task.UDSÆTNINGSGAST_A
- gast #25 does Task.UDKIG
- gast #27 does Task.RORGÆNGER
- gast #29 does Task.VAGTHAVENDE_ELEV
- gast #30 does Task.BJÆRGEMÆRS
- gast #31 does Task.UDSÆTNINGSGAST_C
- gast #32 does Task.ORDONNANS
- gast #35 does Task.DÆKSELEV_I_KABYS
- gast #36 does Task.HU
- gast #37 does Task.UDSÆTNINGSGAST_B
- gast #38 does Task.UDSÆTNINGSGAST_D
Vagt 08 - 12:
- gast #44 does Task.HU
- gast #45 does Task.BJÆRGEMÆRS
- gast #46 does Task.UDSÆTNINGSGAST_A
- gast #47 does Task.DÆKSELEV_I_KABYS
- gast #50 does Task.ORDONNANS
- gast #51 does Task.UDKIG
- gast #53 does Task.RORGÆ

In [289]:
Task(12) == Task.DÆKSELEV_I_KABYS

True